# 第9章 动态规划

## 9.1 背包问题

假设你是个小偷，背着一个可装4磅东西的背包。你可盗窃的商品有如下3件。为了让盗窃的商品价值最高，你该选择哪些商品？

- 吉他 1500美元 1磅
- 音响 3000美元 4磅
- 笔记本电脑 2000美元 3磅

In [1]:
bag_weight = 4
value = [1500,3000,2000] #价值
weight = [1,4,3] #重量

### 9.1.1 简单算法

简单的算法如下：尝试各种可能的商品组合，并找出价值最高的组合。

这样可行，但速度非常慢。在有3件商品的情况下，你需要计算8个不同的集合；有4件商品时，你需要计算16个集合。每增加一件商品，需要计算的集合数都将翻倍！这种算法的运行时间为$ O(2^n)$，真的是慢如蜗牛。

贪婪算法可以找到近似解，但不是最优解，如何找到最优解呢？

### 9.1.2 动态规划

答案是使用动态规划！

下面来看看动态规划算法的工作原理。动态规划先解决子问题，再逐步解决大问题。对于背包问题，你先解决小背包（子背包）问题，再逐步解决原来的问题

动态规划是一个难以理解的概念，如果你没有立即搞懂，也不用担心，我们将研究很多示例。先来演示这种算法的执行过程。看过执行过程后，你心里将有一大堆问题！我将竭尽所能解答这些问题。每个动态规划算法都从一个网格开始，背包问题的网格如下。

![](https://i.loli.net/2019/08/18/JhYjd2qvtb1cXl8.png)

网格的各行为商品，各列为不同容量（1～4磅）的背包。所有这些列你都需要，因为它们将帮助你计算子背包的价值。网格最初是空的。你将填充其中的每个单元格，网格填满后，就找到了问题的答案！你一定要跟着做。请你创建网格，我们一起来填满它。

最终的网格类似于下面这样。

![](https://i.loli.net/2019/08/18/eZbVNxdB6rStF8M.png)


答案如下：将吉他和笔记本电脑装入背包时价值最高，为3500美元。

计算每个单元格的价值时，使用的公式都相同。这个公式如下。

![](https://i.loli.net/2019/08/18/z8DbYVFRLJhgmqd.png)

你可以使用这个公式来计算每个单元格的价值，最终的网格将与前一个网格相同。现在你明白了为何要求解子问题吧？你可以合并两个子问题的解来得到更大问题的解。

In [2]:
def knapsack_dp(value,weight,C):
    n =len(value)
    dp = [[0 for W in range(C+1)] for i in range(n+1)] #多加一行0、一列0
    
    for i in range(1,n+1):
        for W in range(1,C+1):
            if weight[i-1] > W: #当前物品重量大于背包重量，则取i-1个物品时的值
                dp[i][W] = dp[i-1][W]
            else: #当前物品重量小于背包重量，则取两者（不取这个物品、取这个物品）的最大值
                dp[i][W] = max(dp[i-1][W],value[i-1]+dp[i-1][W-weight[i-1]]) #i从1开始，所以是value[i-1],weight[i-1]
    return dp

C = 4 #背包容量
value = [1500,3000,2000] #物品价值
weight = [1,4,3] #物品重量
dp = knapsack_dp(value,weight,C)
dp

[[0, 0, 0, 0, 0],
 [0, 1500, 1500, 1500, 1500],
 [0, 1500, 1500, 1500, 3000],
 [0, 1500, 1500, 2000, 3500]]

## 9.2 背包问题 FAQ

### 9.2.1 再增加一件商品将如何呢
假设你发现还有第四件商品可偷——一个iPhone！

此时需要重新执行前面所做的计算吗？不需要。别忘了，动态规划逐步计算最大价值。到目前为止，背包容量为4磅时，你最多可偷价值3500美元的商品。但这是以前的情况，下面再添加表示iPhone的行。

对于最后一个单元格，情况比较有趣。当前的最大价值为3500美元，但你可偷iPhone，这将余下3磅的容量。

3磅容量的最大价值为2000美元！再加上iPhone价值2000美元，总价值为4000美元。新的最大价值诞生了！

最终的网格如下。

![](https://i.loli.net/2019/08/18/CT2QZDrSgoJMnqy.png)

### 9.2.2 行的排列顺序发生变化时结果将如何

各行的排列顺序无关紧要。

### 9.2.3 可以逐列而不是逐行填充网格吗

自己动手试试吧！就这个问题而言，这没有任何影响，但对于其他问题，可能有影响。

### 9.2.4 增加一件更小的商品将如何呢

假设你还可以偷一条项链，它重0.5磅，价值1000美元。前面的网格都假设所有商品的重量
为整数，但现在你决定把项链给偷了，因此余下的容量为3.5磅。在3.5磅的容量中，可装入的商
品的最大价值是多少呢？不知道！因为你只计算了容量为1磅、2磅、3磅和4磅的背包可装下的商
品的最大价值。现在，你需要知道容量为3.5磅的背包可装下的商品的最大价值。
由于项链的加入，你需要考虑的粒度更细，因此必须调整网格。

![](https://i.loli.net/2019/08/18/bY72QdljwmF6A8k.png)

### 9.2.5 可以偷商品的一部分吗
假设你在杂货店行窃，可偷成袋的扁豆和大米，但如果整袋装不下，可打开包装，再将背包
倒满。在这种情况下，不再是要么偷要么不偷，而是可偷商品的一部分。如何使用动态规划来处
理这种情形呢？

**答案是没法处理**。使用动态规划时，要么考虑拿走整件商品，要么考虑不拿，而没法判断该不该拿走商品的一部分。

但使用贪婪算法可轻松地处理这种情况！首先，尽可能多地拿价值最高的商品；如果拿光了，再尽可能多地拿价值次高的商品，以此类推。

### 9.2.6 旅游行程最优化

假设你要去伦敦度假，假期两天，但你想去游览的地方很多。你没法前往每个地方游览，因此你列个单子。

对于想去游览的每个名胜，都列出所需的时间以及你有多想去看看。根据这个清单，你能确定该去游览哪些名胜吗？

这也是一个背包问题！但约束条件不是背包的容量，而是有限的时间；不是决定该装入哪些商品，而是决定该去游览哪些名胜。请根据这个清单绘制动态规划网格，再接着往下读。

![](https://i.loli.net/2019/08/18/TaxO6i9H2Nv8Wdk.png)

最终的网格如下：

![](https://i.loli.net/2019/08/18/6L5Nr7f2k3njmwR.png)


### 9.2.7 处理相互依赖的情况

假设你还想去巴黎，因此在前述清单中又添加了几项。
去这些地方游览需要很长时间，因为你先得从伦敦前往巴黎，这需要半天时间。如果这3个地方都去玩，是不是要4.5天呢？

不是的，因为不是去每个地方都得先从伦敦到巴黎。到达巴黎后，每个地方都只需1天时间。因此玩这3个地方需要的总时间为3.5天（半天从伦敦到巴黎，每个地方1天），而不是4.5天。

将埃菲尔铁塔加入“背包”后，卢浮宫将更“便宜”：只要1天时间，而不是1.5天。如何使用动态规划对这种情况建模呢？

**没办法建模。动态规划功能强大，它能够解决子问题并使用这些答案来解决大问题。但仅当
每个子问题都是离散的，即不依赖于其他子问题时，动态规划才管用。**这意味着使用动态规划算
法解决不了去巴黎玩的问题。

### 9.2.8 计算最终的解时会涉及两个以上的子背包吗

为获得前述背包问题的最优解，可能需要偷两件以上的商品。但根据动态规划算法的设计，最多只需合并两个子背包，即根本不会涉及两个以上的子背包。不过这些子背包可能又包含子背包。

### 9.2.9 最优解可能导致背包没装满吗
完全可能。假设你还可以偷一颗钻石。这颗钻石非常大，重达3.5磅，价值100万美元，比其他商品都值钱得多。你绝对应该把它给偷了！但当你这样做时，余下的容量只有0.5磅，别的什么都装不下。

## 练习

假设你要去野营。你有一个容量为6磅的背包，需要决定该携带下面的哪些东西。其中
每样东西都有相应的价值，价值越大意味着越重要：

- 水（重3磅，价值10）；
- 书（重1磅，价值3）
- 食物（重2磅，价值9）；
- 夹克（重2磅，价值5）；
- 相机（重1磅，价值6）。

请问携带哪些东西时价值最高？

In [3]:
C = 6 #背包容量
value = [10,3,9,5,6] #物品价值
weight = [3,1,2,2,1] #物品重量
dp = knapsack_dp(value,weight,C)
dp

[[0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 10, 10, 10, 10],
 [0, 3, 3, 10, 13, 13, 13],
 [0, 3, 9, 12, 13, 19, 22],
 [0, 3, 9, 12, 14, 19, 22],
 [0, 6, 9, 15, 18, 20, 25]]

## 9.3 最长公共子串(Longest Common Substring)

- 动态规划可帮助你在给定约束条件下找到最优解。在背包问题中，你必须在背包容量给定的情况下，偷到价值最高的商品。
- 在问题可分解为彼此独立且离散的子问题时，就可使用动态规划来解决。要设计出动态规划解决方案可能很难，这正是本节要介绍的。下面是一些通用的小贴士。
 - 每种动态规划解决方案都涉及网格。
 - 单元格中的值通常就是你要优化的值。在前面的背包问题中，单元格的值为商品的价值。
 - 每个单元格都是一个子问题，因此你应考虑如何将问题分成子问题，这有助于你找出网格的坐标轴。

下面再来看一个例子。假设你管理着网站dictionary.com。用户在该网站输入单词时，你需要给出其定义。

但如果用户拼错了，你必须猜测他原本要输入的是什么单词。

例如，Alex想查单词fish，但不小心输入了hish。在你的字典中，根本就没有这样的单词，但有几个类似的单词。

在这个例子中，只有两个类似的单词，真是太小儿科了。实际上，类似的单词很可能有数千个。

Alex输入了hish，那他原本要输入的是fish还是vista呢？

### 9.3.1 绘制网格

- 单元格中的值是什么？
- 如何将这个问题划分为子问题？
- 网格的坐标轴是什么？

网格如下：

![](https://raw.githubusercontent.com/hostimg/img/gh-pages/s/20190822192521.png)

### 9.3.2 填充网格

我使用下面的公式来计算每个单元格的值。

![](https://raw.githubusercontent.com/hostimg/img/gh-pages/s/20190822192839.png)

实现这个公式的伪代码类似于下面这样。
```python
if word_a[i] == word_b[j]: # 两个字母相同
    cell[i][j] = cell[i-1][j-1] + 1
else: # 两个字母不同
    cell[i][j] = 0
```

查找单词hish和vista的最长公共子串时，网格如下。

![](https://raw.githubusercontent.com/hostimg/img/gh-pages/s/20190822193257.png)

需要注意的一点是，这个问题的最终答案并不在最后一个单元格中！对于前面的背包问题，最终答案总是在最后的单元格中。

但对于最长公共子串问题，答案为**网格中最大的数字**——它可能并不位于最后的单元格中。

**具体实现代码如下：**

可参考[GeeksforGeeks: Longest Common Substring | DP-29](https://www.geeksforgeeks.org/longest-common-substring-dp-29/)

In [4]:
# Python3 implementation of Finding  
# Length of Longest Common Substring  

# Returns length of longest common  
# substring of s1[0..m-1] and s2[0..n-1] 

def LCSubStr(s1,s2):
    # 最长公共子串(Longest Common Substring)
    m = len(s1)
    n = len(s2)
    dp = [[0]*(n+1) for _ in range(m+1)] #初始化，添加一行、一列0 一共m+1行，n+1列
    result = 0  #保存最长公共字串的长度
    for i in range(1,m+1):
        for j in range(1,n+1):
            if s1[i-1] == s2[j-1]: # 相同
                dp[i][j] = dp[i-1][j-1]+1
                result = max(result, dp[i][j])
            else: # 不同
                dp[i][j] = 0
    return result

s1 = [1,3,4,5,6,7,7,8]
s2 = [3,5,7,4,8,6,7,8,2]
LCSubStr(s1,s2)

2

## 9.4 最长公共子序列(Longest Common Subsequence，LCS)

假设Alex不小心输入了fosh，他原本想输入的是fish还是fort呢？如果我们使用最长公共子串公式来比较它们：

![](https://raw.githubusercontent.com/hostimg/img/gh-pages/s/20190822193436.png)

最长公共子串的长度相同，都包含两个字母！但fosh与fish更像。

所以这里应该比较的是**最长公共子序列**：两个单词中**都有的序列**包含的**字母数**。

**最长公共子序列（LCS,Longest Common Subsequence）**：它不要求所求得的字符在所给的字符串中是连续的,而**最长公共子串**则要求字符串是连续的。

如何计算最长公共子序列呢？

### 9.4.1 动态规划算法

下面是填写各个单元格时使用的公式：

![](https://raw.githubusercontent.com/hostimg/img/gh-pages/s/20190822194836.png)

最终的网格如下：

![](https://raw.githubusercontent.com/hostimg/img/gh-pages/s/20190822194437.png)

伪代码如下：

```python
if word_a[i] == word_b[j]:
    cell[i][j] = cell[i-1][j-1] + 1
else:
    cell[i][j] = max(cell[i-1][j], cell[i][j-1])
```

**具体实现代码如下：**

可参考[GeeksforGeeks: Longest Common Subsequence | DP-4](https://www.geeksforgeeks.org/longest-common-subsequence-dp-4/)

In [5]:
# Dynamic Programming implementation of LCS problem 
def LCS(s1,s2):
    # 最长公共子序列(Longest Common Subsequence)
    m = len(s1)
    n = len(s2)
    dp = [[0]*(n+1) for _ in range(m+1)] #初始化，添加一行、一列0 一共m+1行，n+1列
    result = 0  #保存最长公共字串的长度
    for i in range(1,m+1):
        for j in range(1,n+1):
            if s1[i-1] == s2[j-1]: # 相同
                dp[i][j] = dp[i-1][j-1]+1
                result = max(result, dp[i][j])
            else: # 不同
                dp[i][j] = max(dp[i-1][j],dp[i][j-1])
    return dp,result

s1 = [1,3,4,5,6,7,7,8]
s2 = [3,5,7,4,8,6,7,8,2]
dp,result = LCS(s1,s2)
print(result)
dp

5


[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 1, 1, 2, 2, 2, 2, 2, 2],
 [0, 1, 2, 2, 2, 2, 2, 2, 2, 2],
 [0, 1, 2, 2, 2, 2, 3, 3, 3, 3],
 [0, 1, 2, 3, 3, 3, 3, 4, 4, 4],
 [0, 1, 2, 3, 3, 3, 3, 4, 4, 4],
 [0, 1, 2, 3, 3, 4, 4, 4, 5, 5]]

### 9.4.2 递归算法

考虑两个字符串的最后一个字符：

- 1.如果最后一个字符相同，可以转化为均去掉最后一个字符之后的LCS，在加上1
- 2.如果最后一个字符不同，可以转化为：s1去掉最后一个字符和s2的LCS，以及s2去掉最后一个字符和s1的LCS，两者取大值。如：

`LCS(“ABCDGH”, “AEDFHR”) = MAX ( LCS(“ABCDG”, “AEDFHR”), LCS(“ABCDGH”, “AEDFH”) )`

上述朴素递归方法的时间复杂度在最坏情况下为$O(2^n)$ ，当X和Y的所有字符不匹配时，即LCS的长度为0时，发生最坏情况。

**具体代码如下：**

In [6]:
# A Naive recursive Python implementation of LCS problem 
  
def LCS(X, Y, m, n): 
    if m == 0 or n == 0: 
        return 0
    elif X[m-1] == Y[n-1]: 
        return 1 + LCS(X, Y, m-1, n-1)
    else: 
        return max(LCS(X, Y, m, n-1), LCS(X, Y, m-1, n))
  
  
# Driver program to test the above function 
X = "AGGTAB"
Y = "GXTXAYB"
print("Length of LCS is ", LCS(X , Y, len(X), len(Y))) 

Length of LCS is  4


**动态规划的实际应用**：

- 生物学家根据最长公共序列来确定DNA链的相似性，进而判断度两种动物或疾病有多相似。最长公共序列还被用来寻找多发性硬化症治疗方案。
- 你使用过诸如 git diff 等命令吗？它们指出两个文件的差异，也是使用动态规划实现的。
- 前面讨论了字符串的相似程度。编辑距离（levenshtein distance）指出了两个字符串的相似程度，也是使用动态规划计算得到的。编辑距离算法的用途很多，从拼写检查到判断用户上传的资料是否是盗版，都在其中。
- 你使用过诸如Microsoft Word等具有断字功能的应用程序吗？它们如何确定在什么地方断字以确保行长一致呢？使用动态规划！

## 9.5 小结

- 需要在给定约束条件下优化某种指标时，动态规划很有用。
- 问题可分解为离散子问题时，可使用动态规划来解决。
- 每种动态规划解决方案都涉及网格。
- 单元格中的值通常就是你要优化的值。
- 每个单元格都是一个子问题，因此你需要考虑如何将问题分解为子问题。
- 没有放之四海皆准的计算动态规划解决方案的公式。